<a href="https://colab.research.google.com/github/hoodini/roboshaul-adjusted/blob/main/Robo_Shaul_Colab_(Adjusted_by_Yuval_Avidani).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robo-Shaul Colab (Adjusted by Yuval Avidani)

Robo-Shaul project
The Robo-Shaul project was a competition aimed at developing the first open Hebrew text-to-speech engine that replicates the voice of Shaul Amsterdamski, journalist & co-host of the popular Israeli podcast “Hayot Kis”. The competition ran from February, 2023 to May, 2023, and was documented in three episodes, two describing the process leading up to the competition and a final episode with the results.


People
Organizers
Shaul Amsterdamski - journalist & co-host of the Israeli podcast “Hayot Kis”

Orian Sharoni - ML researcher and founder of UpᐧAI consultancy

Roee Shenberg - ML researcher and founder of UpᐧAI consultancy

Uri Elyabayev - Consultant and founder of MDLI community

1st Place
Solution link (GitHub)

Max Melichov
As a versatile, analytical, and resourceful data scientist professional, I bring a wealth of experience in machine learning, data pre-processing, and voice processing techniques. With a proven track record in driving anomaly detection, disease detection, and deepfake algorithm construction, I am driven by the pursuit of organizational growth and thrive on triggering positive change. I champion efficiency, precision, and innovation, constantly seeking new opportunities to push the boundaries of data science.

LinkedIn, Email, Medium

Tony Hasson
A software dev with the goal of achieving expertise in Python, Data (analyzing + management),Automating Workflow and Machine Learning.

Besides that, I’m a problem solver, a great teacher, “out of the box” thinker, a people-person, mostly self taught, philomath, good with numbers and an outer-space geek.

LinkedIn

2nd Place
Solution link (GitHub)

Maxim Lvov
A researcher and algorithms developer in fields of AI, speech recognition and NLP.

LinkedIn

3rd Place
Solution link (GitHub)

Gabriel Khvalevsky
I am an independent and fast learner with a “can do” approach

You don’t require the perfect approval to commence your journey. I didn’t originate from the realm of artificial intelligence, nor did I undergo structured programming training. Nevertheless, I attempted, and I made multiple attempts using diverse approaches. Ultimately, I triumphed, and I take great pride in my accomplishments. In conclusion, I summarize my experience in the finest contest with this phrase: “Success greets those who persistently knock on the door.”

LinkedIn, Email

4th Place
Solution link (Google Colab)

Dr. Eyal Gruss
Code/media/text artist, algorithms researcher, teaches computational creativity at the Holon Institute of Technology.

1. Installation

In [ ]:
#@title Install Tacotron and Waveglow (click to see code)

import os
from os.path import exists, join, basename, splitext
!pip install gdown
git_repo_url = 'https://github.com/maxmelichov/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 2fd4e63
  !pip install -q librosa unidecode
  !pip install Hebrew

import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
import gdown
from hebrew import Hebrew
from hebrew.chars import HebrewChar, ALEPH
from hebrew import GematriaTypes
d = 'https://drive.google.com/uc?id='

#@title Install Tacotron pretrained model  (click to see code)
force_download_TT2 = True
tacotron2_pretrained_model = 'MLPTTS'
if not exists(tacotron2_pretrained_model) or force_download_TT2:
                   # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ PUT MODEL HERE
  gdown.download(d+r'1Vee1F4q59G_YyEdXFz48ZJ7K-KBIincL&export=download', tacotron2_pretrained_model, quiet=False); print("Tacotron2 Model Downloaded")
                   # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑ PUT MODEL HERE

#@title Install Waveglow pretrained (click to see code)
waveglow_pretrained_model = 'waveglow.pt'
if not exists(waveglow_pretrained_model):
  gdown.download(d+r'1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF&export=download', waveglow_pretrained_model, quiet=False); print("WaveGlow Model Downloaded")#1okuUstGoBe_qZ4qUEF8CcwEugHP7GM_b&export

!python ./tacotron2/waveglow/convert_model.py /content/waveglow.pt /content/waveglow.pt

import locale
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding()

#@title Initialize Tacotron and Waveglow and Hebrew to English function
%matplotlib inline
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

graph_width = 900
graph_height = 360
def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
    %matplotlib inline
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='upper',
                       interpolation='none', cmap='inferno')
    fig.canvas.draw()
    plt.show()


import re

 ##deals with . and , in a normal string
def break_to_letter_and_rebuild(string):
    lst=[]
    i=0
    flag=False
    while i<len(string):
        tmp=""
        while i<len(string) and string[i] != '.' and string[i] != ',':
            tmp+=string[i]
            i+=1
            flag=True
        if flag:
            lst.append(tmp)
            flag=False
        if i<len(string):
            lst.append(string[i])
            i+=1

    return lst


##breaks down number to list
def breakdown(number):
    digits = []
    for place, value in zip([100000000,10000000, 1000000,100000, 10000,1000, 100, 10,1], [100000000,10000000, 1000000,100000, 10000,1000, 100, 10,1]):
        digit = number // place * value
        digits.append(digit)
        number -= digit
    return digits

##auxilary function for NumberToHebrew , helps break down arrays of 3's
def build_three_num_heb(list_num,num_dict_below_20,num_dict_eq_above_20,last):

    flag_zero=1
    list_heb=[]
    for i, num in enumerate(list_num):
        if num == 0:
            continue
        else:
            flag_zero=0
            if i < 1:
                list_heb.append(num_dict_eq_above_20[num])

            elif i == 1:
                if list_num[0] != 0:
                    if list_num[1] + list_num[2] < 20:
                        list_heb.append("וְ" + num_dict_below_20[list_num[1] + list_num[2]])
                        break
                    else:
                        list_heb.append(num_dict_eq_above_20[num])
                else:
                    if list_num[1] + list_num[2] < 20:
                        list_heb.append(num_dict_below_20[list_num[1] + list_num[2]])
                        break
                    else:
                        list_heb.append(num_dict_eq_above_20[num])

            elif i == 2:
                if list_num[0] != 0 or list_num[1] != 0 or last:
                    list_heb.append("וְ" + num_dict_below_20[num])
                else:
                    list_heb.append(num_dict_below_20[num])

    return list_heb,flag_zero

##gets number and turns it into hebrew with nikud
def NumberToHebrew(number):

    if number==0:
        return ["אֶפֶס"]

    signs_dict = {
        '%': 'אָחוּז',
        ',': 'פְּסִיק',
        '.': 'נְקֻדָּה'

    }

    num_dict_below_20={
        1: 'אֶחָד',
        2: 'שְׁנַיִם',
        3: 'שְׁלֹשָׁה',
        4: 'אַרְבָּעָה',
        5: 'חֲמִשָּׁה',
        6: 'שִׁשָּׁה',
        7: 'שִׁבְעָה',
        8: 'שְׁמוֹנָה',
        9: 'תִּשְׁעָה',
        10: 'עֶשֶׂר',
        11: 'אַחַד עָשָׂר',
        12: 'שְׁנֵים עָשָׂר',
        13: 'שְׁלֹשָׁה עָשָׂר',
        14: 'אַרְבָּעָה עָשָׂר',
        15: 'חֲמִשָּׁה עָשָׂר',
        16: 'שִׁשָּׁה עָשָׂר',
        17: "שִׁבְעָה עֶשְׂרֵה",
        18: "שְׁמוֹנָה עֶשְׂרֵה",
        19: "תִּשְׁעָה עֶשְׂרֵה"
    }

    num_dict_eq_above_20={
        20: "עֶשְׂרִים",
        30: "שְׁלשִׁים",
        40: "אַרְבָּעִים",
        50: "חֲמִשִּׁים",
        60: "שִׁשִּׁים",
        70: "שִׁבְעִים",
        80: "שְׁמוֹנִים",
        90: "תִּשְׁעִים",
        100: "מֵאָה",
        200: "מָאתַיִם",
        300: "שְׁלֹשׁ מֵאוֹת",
        400: "אַרְבָּעִ מֵאוֹת",
        500: "חֲמִשֶּׁ מֵאוֹת",
        600: "שֵׁשׁ מֵאוֹת",
        700: "שִׁבְעַ מֵאוֹת",
        800: "שְׁמוֹנֶ מֵאוֹת",
        900: "תִּשְׁעַ מֵאוֹת",
        1000: "אֶלֶף",
        2000: "אֲלַפַּיִם",
        3000: "שְׁלֹשֶׁת אֲלָפִים",
        4000: "אַרְבַּעַת אֲלָפִים",
        5000: "חֲמֵשׁ אֲלָפִים",
        6000: "שֵׁשׁ אֲלָפִים",
        7000: "שִׁבְעָה אֲלָפִים",
        8000: "שְׁמוֹנָה אֲלָפִים",
        9000: "תִּשְׁעָה אֲלָפִים"
    }

    if number in signs_dict:
        return [signs_dict[number]]

    if number<10000:

        list_heb=[]
        list_num=breakdown(number)
        list_num=list_num[5:]
        for i,num in enumerate(list_num):
            if num==0:
                continue
            else:
                if i<2:
                    list_heb.append(num_dict_eq_above_20[num])

                elif i==2:
                    if list_num[0]!=0 or list_num[1]!=0:
                        if list_num[2]+list_num[3]<20:
                            list_heb.append("וְ"+num_dict_below_20[list_num[2]+list_num[3]])
                            break
                        else:
                            list_heb.append(num_dict_eq_above_20[num])
                    else:
                        if list_num[2] + list_num[3] < 20:
                            list_heb.append(num_dict_below_20[list_num[2] + list_num[3]])
                            break
                        else:
                            list_heb.append(num_dict_eq_above_20[num])

                elif i==3:
                    if list_num[0]!=0 or list_num[1]!=0 or list_num[2]!=0:
                        list_heb.append("וְ" + num_dict_below_20[num])
                    else:
                        list_heb.append(num_dict_below_20[num])

        return list_heb

    else:

        list_heb = []
        list_num = breakdown(number)
        s1,s2,s3=list_num[:3],list_num[3:6],list_num[6:]



        ##take care of millions

        # set them up for transcript
        for i in range(len(s1)):
            s1[i]=s1[i]/1000000

        ret_list,flag_zero=build_three_num_heb(s1,num_dict_below_20,num_dict_eq_above_20,False)
        if not flag_zero:
            for item in ret_list:
                list_heb.append(item)
            list_heb.append("מִילְיוֹן")

        ##take care of thousands

        # set them up for transcript
        for i in range(len(s2)):
            s2[i] = s2[i] / 1000

        ret_list, flag_zero = build_three_num_heb(s2, num_dict_below_20, num_dict_eq_above_20,False)
        if not flag_zero:
            for item in ret_list:
                list_heb.append(item)
            list_heb.append("אֶלֶף")

        ##take care of hundred and leftovers
        ret_list, flag_zero = build_three_num_heb(s3, num_dict_below_20, num_dict_eq_above_20,True)
        if not flag_zero:
            for item in ret_list:
                list_heb.append(item)

        return list_heb



##attempts to split string to number and string
def split_number_and_string(input_string):
    # Use regular expression to find any number within the string
    match = re.search(r'\d+', input_string)

    if match:
        # Extract the number from the string
        number = match.group()

        # Split the string into two parts: before and after the number
        index = match.start()
        string_before_number = input_string[:index]
        string_after_number = input_string[index + len(number):]

        return string_before_number, number, string_after_number
    else:
        # If no number is found, return None
        return None

#######################################################################Auxilary functions



##check if string has number in it
def has_number(input_string):
    # Use regular expression to search for any digits within the string
    return bool(re.search(r'\d', input_string))

##breaks text to list
def break_to_list(text):
    """
    This function receives a string and returns a list of strings with each word from the input text.
    """
    lst = []
    for tav in text:
        lst.append(tav)
    return lst

####################################


########################## relevant for fixing text with numbers:

def is_number_with_comma(string):
    # Remove trailing comma, if any
    if string.endswith(',') or string.endswith('.'):
        string = string[:-1]

    # Check if string matches pattern
    if ',' in string:
        parts = string.split(',')
        if len(parts) != 2:
            return False
        if not all(part.isdigit() for part in parts):
            return False
    elif not string.isdigit():
        return False

    return True

def clean_number_with_comma(string):
    # Remove trailing comma, if any
    if string.endswith(',') or string.endswith('.'):
        string = string[:-1]

    # Remove commas from string
    string = string.replace(',', '')

    # Convert string to integer and return
    return int(string)


def is_number_with_decimal(string):
    if ',' in string:
        if string[-1] != ',':
            return False
        string = string[:-1]
    if '.' not in string:
        return False
    try:
        float(string)
    except ValueError:
        return False
    return True

def clean_decimal(string):
    if ',' in string:
        if string[-1] != ',':
            return None
        string = string[:-1]

    parts = string.split('.')
    try:
        return int(parts[0]),int(parts[1])
    except:
        return int(parts[0]),None


def is_percentage(string):

    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]

    if not string.endswith('%'):
        return False
    string = string[:-1]
    if string.endswith(','):
        string = string[:-1]
    try:
        float(string)
    except ValueError:
        return False
    return True


def clean_percentage(string):
    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]

    if string.endswith('%'):
        string = string[:-1]
    else:
        return None
    if ',' in string:
        if string[-1] != ',':
            return None
        string = string[:-1]
    try:
        number = float(string)
    except ValueError:
        return None
    return str(number).rstrip('0').rstrip('.')

def is_number_range(string):


    if '-' not in string:
        return False

    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]


    parts = string.split('-')
    if len(parts) != 2:
        return False
    for part in parts:
        if not part.isdigit():
            return False
    return True

def clean_number_range(string):
    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]

    parts = string.split('-')
    return (int(parts[0]), int(parts[1]))


def is_pattern_number_with_heb(string):
    if '-' not in string:
        return False

    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]

    parts = string.split('-')

    if not parts[1].isdigit() and not is_number_range(parts[1]) and not is_number_with_comma(parts[1]) and not is_number_with_decimal(parts[1]) and not is_percentage(parts[1]):
            return False

    return True

def clean_pattern_number_with_heb(string):
    if string[-1] == ',' or string[-1] == '.':
        string = string[:-1]

    parts = string.split('-')
    if len(parts)<=2:
        ## '4,000,' / '10,000' / '1000,'
        if is_number_with_comma(parts[1]):
            return parts[0],str(clean_number_with_comma(parts[1])) , "is_number_with_comma"

        ## '2.9' / '3.4'
        elif is_number_with_decimal(parts[1]):
            return parts[0],clean_decimal(parts[1]) , "is_number_with_decimal"

        ## '4.5%' / '9.25%.' / '26.5%,'
        elif is_percentage(parts[1]):
            return parts[0],str(clean_percentage(parts[1])) , "is_percentage"

    ## '5-6' / '1971-1972,' / '2003-2005.'
    if len(parts)>2 and is_number_range(parts[1]+'-'+parts[2]):
        return parts[0],(str(parts[1]),str(parts[2])) , "is_number_range"



def clean_number(word):

    ## '4,000,' / '10,000' / '1000,'
    if is_number_with_comma(word):
        return NumberToHebrew(int(clean_number_with_comma(word)))

    ## '2.9' / '3.4'
    elif is_number_with_decimal(word):
        list_heb=[]
        part1,part2=clean_decimal(word)
        list_heb+=NumberToHebrew(part1)
        list_heb+=NumberToHebrew('.')
        list_heb+=NumberToHebrew(part2)
        return list_heb

    ## '4.5%' / '9.25%.' / '26.5%,'
    elif is_percentage(word):
        list_heb = []
        part1, part2 = clean_decimal(clean_percentage(word))

        if part2!=None:
            list_heb += NumberToHebrew(part1)
            list_heb += NumberToHebrew('.')
            list_heb += NumberToHebrew(part2)
            list_heb += NumberToHebrew('%')
            return list_heb
        else:
            list_heb += NumberToHebrew(part1)
            list_heb += NumberToHebrew('%')
            return list_heb

    ## '5-6' / '1971-1972,' / '2003-2005.'
    elif is_number_range(word):
        list_heb = []
        part1, part2 = clean_number_range(word)
        list_heb += NumberToHebrew(part1)
        list_heb.append("עַד")
        list_heb += NumberToHebrew(part2)
        return list_heb

    ##    'בְּ-100,000'   / בְּ-99.99%  / הַ-1,100  /   מִ-0.7%   /  לְ-1.9    /  כְּ-22,000.
    elif is_pattern_number_with_heb(word):
        heb_letter,num,func=(clean_pattern_number_with_heb(word))
        #arr_attr= (clean_pattern_number_with_heb(word))
        list_heb = []
        list_heb.append(heb_letter)

        if func=="is_number_with_comma":
            list_heb+=NumberToHebrew(int(num))
            return list_heb

        elif func=="is_number_with_decimal":
            part1, part2 = num
            list_heb += NumberToHebrew(part1)
            list_heb += NumberToHebrew('.')
            list_heb += NumberToHebrew(part2)
            return list_heb

        elif func=="is_percentage":
            part1, part2 = clean_decimal(num)

            if part2 != None:
                list_heb += NumberToHebrew(part1)
                list_heb += NumberToHebrew('.')
                list_heb += NumberToHebrew(part2)
                list_heb += NumberToHebrew('%')
                return list_heb
            else:
                list_heb += NumberToHebrew(part1)
                list_heb += NumberToHebrew('%')
                return list_heb

        elif func == "is_number_range":
            part1, part2 = num
            list_heb += NumberToHebrew(int(part1))
            list_heb.append("עַד")
            list_heb += NumberToHebrew(int(part2))
            return list_heb

#######################################################

##takes a letter in hebrew and returns the sound in english
def HebrewLetterToEnglishSound(obj,tzuptzik,last_letter=False):
    obj = Hebrew(obj).string
    # map the nikud symbols to their corresponding phenoms
    nikud_map = {"ָ": "a", "ַ": "a", "ֶ": "e", "ֵ": "e", "ִ": "i", "ְ": "", "ֹ": "o", "ֻ": "oo", 'ּ': "", 'ֲ': 'a'}


    beged_kefet_shin_sin = {
        ############ B
        "בּ": "b",
        "בְּ": "b",
        "בִּ": "bi",
        "בֹּ": "bo",
        "בֵּ": "be",
        "בֶּ": "be",
        "בַּ": "ba",
        "בָּ": "ba",
        "בֻּ": "boo",
        ############ G
        "גּ": "g",
        "גְּ": "g",
        "גִּ": "gi",
        "גֹּ": "go",
        "גֵּ": "ge",
        "גֶּ": "ge",
        "גַּ": "ga",
        "גָּ": "ga",
        "גֻּ": "goo",
        ########### D
        "דּ": "d",
        "דְּ": "d",
        "דִּ": "di",
        "דֹּ": "do",
        "דֵּ": "de",
        "דֶּ": "de",
        "דַּ": "da",
        "דָּ": "da",
        "דֻּ": "doo",
        ########### K
        "כּ": "k",
        "כְּ": "k",
        "כִּ": "ki",
        "כֹּ": "ko",
        "כֵּ": "ke",
        "כֶּ": "ke",
        "כַּ": "ka",
        "כָּ": "ka",
        "כֻּ": "koo",
        ############ P
        "פּ": "p",
        "פְּ": "p",
        "פִּ": "pi",
        "פֹּ": "po",
        "פֵּ": "pe",
        "פֶּ": "pe",
        "פַּ": "pa",
        "פָּ": "pa",
        "פֻּ": "poo",
        ############ T
        "תּ": "t",
        "תְּ": "t",
        "תִּ": "ti",
        "תֹּ": "to",
        "תֵּ": "te",
        "תֶּ": "te",
        "תַּ": "ta",
        "תָּ": "ta",
        "תֻּ": "too",
        ############ S
        "שׂ": "s",
        "שְׂ": "s",
        "שִׂ": "si",
        "שֹׂ": "so",
        "שֵׂ": "se",
        "שֶׂ": "se",
        "שַׂ": "sa",
        "שָׂ": "sa",
        "שֻׂ": "soo",
        ########### SH
        "שׁ": "sh",
        "שְׁ": "sh",
        "שִׁ": "shi",
        "שֹׁ": "sho",
        "שֵׁ": "she",
        "שֶׁ": "she",
        "שַׁ": "sha",
        "שָׁ": "sha",
        "שֻׁ": "shoo",
    }

    vav = {
        "וֵּו": "ve",
        "וּ": "oo",
        "וּו": "oo",
        "וֹ": "o",
        "וֹו": "oo",
        "וְ": "ve",
        "וֱו": "ve",
        "וִ": "vi",
        "וִו": "vi",
        "וַ": "va",
        "וַו": "va",
        "וֶ": "ve",
        "וֶו": "ve",
        "וָ": "va",
        "וָו": "va",
        "וֻ": "oo",
        "וֻו": "oo"
    }


    letters_map = {
        "א": "",
        "ב": "v",
        "ג": "g",
        "ד": "d",
        "ה": "hh",
        "ו": "v",
        "ז": "z",
        "ח": "h",
        "ט": "t",
        "י": "y",
        "כ": "h",
        "ל": "l",
        "מ": "m",
        "נ": "n",
        "ס": "s",
        "ע": "",
        "פ": "f",
        "צ": "ts",
        "ק": "k",
        "ר": "r",
        "ש": "sh",
        "ת": "t",
        "ן": "n",
        "ם": "m",
        "ף": "f",
        "ץ": "ts",
        "ך": "h",
    }

    patah_ganav={
        "חַ": "ah",
        "חָ": "ah",
        "הַ": "hha",
        "הָ": "hha",
        "עַ": "a",
        "עָ": "a",

    }

    tzuptzik_letters={
        ##G
        "ג": "j",
        "גְ": "j",
        "גִ": "ji",
        "גֹ": "jo",
        "גֵ": "je",
        "גֶ": "je",
        "גַ": "ja",
        "גָ": "ja",
        "גֻ": "joo",
        "גּ": "j",
        "גְּ": "j",
        "גִּ": "ji",
        "גֹּ": "jo",
        "גֵּ": "je",
        "גֶּ": "je",
        "גַּ": "ja",
        "גָּ": "ja",
        "גֻּ": "joo",

        ##ch
        "צ": "ch",
        "צְ": "ch",
        "צִ": "chi",
        "צֹ": "cho",
        "צֵ": "che",
        "צֶ": "che",
        "צַ": "cha",
        "צָ": "cha",
        "צֻ": "choo",

        ##ch
        "ץ": "ch",
        "ץְ": "ch",
        "ץִ": "chi",
        "ץֹ": "cho",
        "ץֵ": "che",
        "ץֶ": "che",
        "ץַ": "cha",
        "ץָ": "cha",
        "ץֻ": "choo",

        ##Z
        "ז": "zh",
        "זְ": "zh",
        "זִ": "zhi",
        "זֹ": "zho",
        "זֵ": "zhe",
        "זֶ": "zhe",
        "זַ": "zha",
        "זָ": "zha",
        "זֻ": "zhoo",
    }

    if last_letter:
        if obj in patah_ganav:
            return patah_ganav[obj]

    if tzuptzik==True:
        if obj in tzuptzik_letters:
            return tzuptzik_letters[obj]

    if obj in beged_kefet_shin_sin:
        return beged_kefet_shin_sin[obj]
    elif obj in vav:
        return vav[obj]
    else:
        lst = break_to_list(obj)
        string = ""
        for item in lst:
            if item in letters_map:
                string += letters_map[item]
            if item in nikud_map:
                string += nikud_map[item]

        return string


##takes hebrew word and turns it into the sound in english
def HebrewWordToEnglishSound(word,index):
    new_sentence=""
    hs = Hebrew(word)
    hs = Hebrew(list(hs.graphemes)).string
    for i, letter in enumerate(hs):

        tzuptzik = False
        if i < len(hs) - 1:
            if hs[i + 1] == '\'':
                tzuptzik = True

        tav = HebrewLetterToEnglishSound(letter, tzuptzik, i == len(hs) - 1)
        new_sentence += tav

    ##clean list:
    try:
        if new_sentence[-1] == 'y' and new_sentence[-2] == 'y':
            new_sentence = new_sentence.replace("yy", "y")
    except:
        pass
    return new_sentence

##takes hebrew sentence and turns it into english sounds
def ARPA(sentence,index=0):
    words = sentence.split()
    new_sentence = ""

    for word in words:
        ##if number not in string
        if not has_number(word):

            ##breaks the word to letters and ',' and '.'
            broken_word=break_to_letter_and_rebuild(word)

            for brk_word in broken_word:

                ##tries to add silence
                if brk_word=='.' or brk_word==',' or brk_word==';':
                    new_sentence += "q"+" "

                else:
                    ret_sentence=HebrewWordToEnglishSound(brk_word,index)
                    new_sentence+=ret_sentence+" "

        ##if there is a number:
        else:
            try:
                before_num,num,after_num=split_number_and_string(word)

                if has_number(after_num) or has_number(before_num):
                    list_of_numbers=clean_number(word)
                    for number in list_of_numbers:
                        ret_sentence = HebrewWordToEnglishSound(number, index)
                        new_sentence += ret_sentence + " "

                else:
                    ret_sentence = HebrewWordToEnglishSound(before_num, index)
                    new_sentence += ret_sentence+" "

                    num = [s for s in word if s.isdigit()]
                    num="".join(num)
                    num=int(num)
                    list_of_numbers=NumberToHebrew(num)
                    for number in list_of_numbers:
                        ret_sentence=HebrewWordToEnglishSound(number,index)
                        new_sentence += ret_sentence + " "

                    ret_sentence = HebrewWordToEnglishSound(after_num, index)
                    new_sentence += ret_sentence + " "



            except:
                print("error from split_number_and_string in line:", index,"with word: ",word)



    return new_sentence

torch.set_grad_enabled(False)

# initialize Tacotron2 with the pretrained model
hparams = create_hparams()

# Load Tacotron2 (run this cell every time you change the model)
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1800 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval()

# Load WaveGlow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

# 2. Text Input to the Robo-Shaul Hebrew TTS voice model

In [ ]:

#This is the text that will be created as audio, feel free to change it

text = """
אָז שָׁלוֹם חֲבֵרִים, אֲנִי יוּבָל מֵהָאק אֵט עֲרוּץ פּוֹדְקַאסְט וּבְלֹג בִּינָה מְלָאכוּתִית וְהָאקִינְג בְּעִבְרִית וְהַסִּרְטוֹן הַזֶּה יִהְיֶה מְאוֹד מְאוֹד זָרִיז
אֲנִי רוֹצֶה לְהַרְאוֹת לָכֶם אֵיךְ שִׁמּוּשׁ בְּאוֹטוֹ גִ'י פִּי טִי יָכוֹל לַחֲסֹךְ לָנוּ הַרְבֵּה מְאוֹד עֲבוֹדָה.
אָז כָּאן מָה שֶׁאַתֶּם רוֹאִים זֶה הַמָּסָךְ שֶׁל אוֹטוֹ גִ'י פִּי טִי,
טִישֶׁיֵּשׁ לִי סִרְטוֹן אַחֵר שֶׁכְּבָר הִדְגַּמְתִּי אֵיךְ אֲנַחְנוּ מַתְקִינִים אוֹתוֹ  אֲנִי בְּעֶצֶם מַגְדִּיר אֶת הַמַּטָּרוֹת שֶׁלִּי,
עַכְשָׁו שָׁמַרְתִּי אוֹתָם כָּאן בַּצַּד אָז אֲנִי רַק אַעְתִּיק אוֹתָם לְרֶגַע,
אֲנִי בְּעֶצֶם אוֹמֵר לָאוֹטוֹ גִ'י פִּי טִי לִיצָר לִי עַמּוּד נְחִיתָה שֶׁבּוֹ אֲנִי רוֹצֶה לִמְכֹּר קוּרְס שֶׁל בִּינָה מְלָאכוּתִית,
אֲנִי מְנַסֶּה לְהַסְבִּיר לוֹ לְהִשְׁתַּמֵּשׁ בְּקוֹד
"""

# 3. Run to get the audio file

In [ ]:
sigma = 0.8
denoise_strength = 0.2
# try to switch raw data to True maybe the results will be better
raw_input = False # disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing

for i in text.split("\n"):
    if len(i) < 1: continue;
    print(i)
    if raw_input:
        if i[-1] != ";": i=i+";"
    else: i = ARPA(i)
    print(i)
    with torch.no_grad(): # save VRAM by not including gradients
        sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
        sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
        mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
        plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],alignments.float().data.cpu().numpy()[0].T))
        audio = waveglow.infer(mel_outputs_postnet, sigma=sigma); print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))